In [2]:
%env PGINSTANCE=pg

env: PGINSTANCE=pg


In [3]:
%env $BASE_DIR=/home

env: $BASE_DIR=/home


In [4]:
!gsutil cp gs://cs327e-open-access-2/shopify.zip .

Copying gs://cs327e-open-access-2/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [5]:
!unzip -o shopify.zip

Archive:  shopify.zip
  inflating: __MACOSX/._shopify      
  inflating: shopify/apps_categories.csv  
  inflating: __MACOSX/shopify/._apps_categories.csv  
  inflating: shopify/categories.csv  
  inflating: __MACOSX/shopify/._categories.csv  
  inflating: shopify/reviews.csv     
  inflating: __MACOSX/shopify/._reviews.csv  
  inflating: shopify/.DS_Store       
  inflating: __MACOSX/shopify/._.DS_Store  
  inflating: shopify/key_benefits.csv  
  inflating: __MACOSX/shopify/._key_benefits.csv  
  inflating: shopify/pricing_plan_features.csv  
  inflating: __MACOSX/shopify/._pricing_plan_features.csv  
  inflating: shopify/pricing_plans.csv  
  inflating: __MACOSX/shopify/._pricing_plans.csv  
  inflating: shopify/apps.csv        
  inflating: __MACOSX/shopify/._apps.csv  


In [155]:
# get apps by category

from google.cloud import firestore
db = firestore.Client()
import pandas as pd
batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

# print(rows)

df2 = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
rows2 = df2.values.tolist()

for row in rows:
    record = {}
    record['id'] = row[0]
    record['title'] = row[1]
    
    catRef = db.collection('category').document(row[0])
    batch.set(catRef, record)
    
    for row2 in rows2:
        if str(row[0]) == str(row2[1]): 
            record2 = {}
            record2['app_id'] = row2[0]
            record2['category_id'] = row2[1]
            
            appCatRef = db.collection('appsCat').document(row2[0])
            
            batch.set(appCatRef, record2)
            batch.commit()

In [156]:
# get apps with highest review_count

from google.cloud import firestore
db = firestore.Client()
import pandas as pd
batch = db.batch()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows:
    record = {}
    record['id'] = row[0]
    record['ur'] = row[1]
    record['title'] = row[2]
    record['developer'] = row[3]
    record['developer_link'] = row[4]
    record['icon'] = row[5]
    record['rating'] = row[6]
    record['review_count'] = row[7]
    
    appsRef = db.collection('apps').document(row[0])
    batch.set(appsRef,record)
    
    batch.commit()

In [157]:
# get pricing plan details by app

from google.cloud import firestore
db = firestore.Client()
import pandas as pd
batch = db.batch()

df1 = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
rows1 = df1.values.tolist()

df2 = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
rows2 = df2.values.tolist()

for row in rows1:
    record = {}
    record['id'] = row[0]
    record['ur'] = row[1]
    record['title'] = row[2]
    record['developer'] = row[3]
    record['developer_link'] = row[4]
    record['icon'] = row[5]
    record['rating'] = row[6]
    record['review_count'] = row[7]
    
    appsRef = db.collection('appsPlan').document(row[0])
    batch.set(appsRef, record)
    
    for row2 in rows2:
        if row[0] == row2[1]:
            record2 = {}
            record2['app_id'] = row2[0]
            record2['pricing_plan_id'] = row2[1]
            record2['feature'] = row2[2]
            
            pricingRef = db.collection('pricing_plan').document(row2[1])
            
            batch.set(pricingRef, record2)
            batch.commit()

In [158]:
# get key benefits by app

from google.cloud import firestore
db = firestore.Client()
import pandas as pd
batch = db.batch()

df1 = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
rows1 = df1.values.tolist()
df2 = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
rows2 = df2.values.tolist()

for row in rows1:
    record = {}
    record['id'] = row[0]
    record['ur'] = row[1]
    record['title'] = row[2]
    record['developer'] = row[3]
    record['developer_link'] = row[4]
    record['icon'] = row[5]
    record['rating'] = row[6]
    record['review_count'] = row[7]
    
    appsRef = db.collection('appsBenefits').document(row[0])
    batch.set(appsRef, record)
    
    for row2 in rows2:
        if row[0] == row2[0]:
            record2 = {}
            record2['app_id'] = row2[0]
            record2['title'] = row2[1]
            record2['description'] = row2[2]
            
            benRef = db.collection('key_benefits').document(row2[0])
            
            batch.set(benRef, record2)
            batch.commit()

In [33]:
#get count of documents in each collection
from google.cloud import firestore 
db = firestore.Client()

catapps = db.collection('appsCat').stream()
cat = db.collection('category').stream()
apps = db.collection('apps').stream()
priceapps = db.collection('appsPlan').stream()
price = db.collection('pricing_plan').stream()
keyapps = db.collection('appsBenefits').stream()
key = db.collection('key_benefits').stream()

#category apps collection

count = 0
for res in catapps:
    count += 1
print("The number of documents in the appsCat collection is: " + str(count))
count = 0 

#category collection
for res in cat:
    count += 1
print ("The number of documents in the category collection is: " + str(count))
count = 0

#apps collection
for res in apps:
    count += 1
print ('The number of documents in the apps collection is: ' + str(count))
count = 0

#price apps collection
for res in priceapps:
    count += 1
print ('The number of documents in the appsPlan collection is: ' + str(count))
count = 0

#price collection
for res in price:
    count += 1
print ('The number of documents in the pricing_plan collection is: ' + str(count))
count = 0

#key apps collection
for res in keyapps:
    count += 1
print ('The number of documents in the appsBenefits collection is: ' + str(count))
count = 0

#key benefit collection
for res in key:
    count += 1
print ('The number of documents in the key_benefits collection is: ' + str(count))
count = 0

The number of documents in the appsCat collection is: 3547
The number of documents in the category collection is: 12
The number of documents in the apps collection is: 3547
The number of documents in the appsPlan collection is: 3547
The number of documents in the pricing_plan collection is: 3669
The number of documents in the appsBenefits collection is: 3547
The number of documents in the key_benefits collection is: 3184


In [32]:
# get top 10 productivity apps
from google.cloud import firestore
db = firestore.Client()

# get productivity catID
docs = db.collection(u'category').where('title','==', 'Productivity').stream()
for doc in docs:
    my_dict = doc.to_dict()
    prod_id = my_dict['id']

#get app ids
prod_app_id = db.collection('appsCat').where('category_id', '==', prod_id).stream()
id_list = []
for doc in prod_app_id:
    my_dict2 = doc.to_dict()
    if (len(id_list)) < 30:
        id_list.append(my_dict2['app_id'])
    else:
        break

#get apps
prod_one = db.collection('apps').where('id', 'in', id_list).order_by('rating', direction = firestore.Query.DESCENDING).limit(10).stream()
for doc in prod_one:
    #print(f'{doc.id} => {doc.to_dict()}')
    my_dict3 = doc.to_dict()
    if my_dict3['rating'] in real_max_list:
        print('ID: ' + str(my_dict3['id']) + '\nTitle: ' + str(my_dict3['title']) + '\nDeveloper: ' + str(my_dict3['developer']) + '\nRating: ' + str(my_dict3['rating']) +'\nReview Count: ' + str(my_dict3['review_count']) + '\n')

ID: 9054b3e5-9405-41a0-a6f8-713ac1530220
Title: Auto Schedule
Developer: JD
Rating: 5.0
Review Count: 1

ID: 84c8cf7e-4997-4fd2-940b-0c642f890ac6
Title: Collection Sort
Developer: pluginappstore
Rating: 5.0
Review Count: 41

ID: 62e5a4ba-0671-4abf-8478-6886ab0b9509
Title: Admin+
Developer: Modd Apps Inc.
Rating: 5.0
Review Count: 8

ID: 5cfd4958-674e-41e3-b850-b0b7716f8d1a
Title: Sort Orders by Power Tools
Developer: Power Tools
Rating: 5.0
Review Count: 5

ID: 5a80c64a-92f7-44d4-812c-0a8c2d10eb7f
Title: API Helper
Developer: William Belk
Rating: 5.0
Review Count: 2

ID: 5952ad54-efc9-4cc7-978d-44771606496d
Title: Mechanic
Developer: Lightward
Rating: 5.0
Review Count: 41

ID: 58a6a8ca-4559-4664-a198-c0854e9b0f93
Title: High Conversion Collections
Developer: Tagalys
Rating: 5.0
Review Count: 1

ID: 4d0827e7-ab62-467a-91a7-56a41d3f48f4
Title: Auto Hide Sold‑out Products
Developer: Zoocommerce
Rating: 5.0
Review Count: 11

ID: 4c5958ef-0e1d-4f9f-a7c9-466f22fba545
Title: Old Product Colle

In [15]:
from google.cloud import firestore
db = firestore.Client()
apps = db.collection('apps')
query = apps.order_by('review_count', direction = firestore.Query.DESCENDING).limit(5)
result = query.stream()
for doc in result:
    my_dict = doc.to_dict()
    print ('ID: ' + str(my_dict['id']) + '\nTitle: ' + str(my_dict['title']) + '\nDeveloper: ' + str(my_dict['developer']) + '\nRating: ' + str(my_dict['rating']) +'\nReview Count: ' + str(my_dict['review_count']) + '\n')

ID: d9f142ee-b141-4dc4-9353-173db61d2eb0
Title: Privy ‑ Exit Pop Ups & Email
Developer: Privy
Rating: 4.7
Review Count: 23078

ID: 78ea0810-c008-4a4e-a82f-de0c790e3286
Title: Free Shipping Bar
Developer: Hextom
Rating: 4.9
Review Count: 8737

ID: b88488b0-9912-44d3-b736-224c36f09d95
Title: Sales Pop ‑ Popup Notification
Developer: CartKit
Rating: 4.8
Review Count: 6905

ID: e528a60e-94f8-4e92-80e2-5bc6013b8283
Title: BEST Currency Converter
Developer: Grizzly Apps
Rating: 4.8
Review Count: 5986

ID: be2640c4-01b5-4d52-9f68-cae8c0734d0d
Title: Recart FB Messenger Marketing
Developer: Recart
Rating: 4.8
Review Count: 5596

